In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out


openjdk-8-jdk-headless is already the newest version (8u432-ga~us1-0ubuntu2~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import numpy as np
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [3]:
G =  sc.textFile("graph-full.txt")
n = 1000

In [4]:
beta = 0.8

**Part (a):**

In [5]:
edges = G.map(lambda x: (int(x.split("\t")[0]) - 1, [int(x.split("\t")[1]) - 1])).reduceByKey(lambda a, b: a + b).map(lambda x: (x[0], list(set(x[1]))))

In [6]:
r = np.ones(n) / n

In [7]:
for _ in range(40):
  new_r = np.zeros(n)
  for source, targets in edges.toLocalIterator():
    entry = 1 / len(targets)
    for target in targets:
      new_r[target] += entry * r[source]
  r = beta * new_r + (1 - beta) / n


In [8]:
print("Top 5 node ids with PageRank scores:")
for index in np.argsort(-r)[0:5]:
  print("Node id ", index + 1, "has PageRank score ", r[index])

Top 5 node ids with PageRank scores:
Node id  263 has PageRank score  0.0020202911815182184
Node id  537 has PageRank score  0.0019433415714531492
Node id  965 has PageRank score  0.0019254478071662627
Node id  243 has PageRank score  0.001852634016241731
Node id  285 has PageRank score  0.001827372170064514


In [9]:
print("Bottom 5 node ids with PageRank scores:")
for index in np.argsort(r)[0:5]:
  print("Node id ", index + 1, "has PageRank score ", r[index])

Bottom 5 node ids with PageRank scores:
Node id  558 has PageRank score  0.0003286018525215297
Node id  93 has PageRank score  0.0003513568937516577
Node id  62 has PageRank score  0.00035314810510596274
Node id  424 has PageRank score  0.00035481538649301454
Node id  408 has PageRank score  0.00038779848719291705


**Part (b):**

In [10]:
h = np.ones(n)

In [11]:
for _ in range(40):
  new_a = np.zeros(n)
  new_h = np.zeros(n)
  for source, targets in edges.toLocalIterator():
    for target in targets:
      new_a[target] += h[source]
  a = new_a / np.max(new_a)
  for source, targets in edges.toLocalIterator():
    for target in targets:
      new_h[source] += a[target]
  h = new_h / np.max(new_h)

In [12]:
print("Top 5 node ids with hubbiness scores:")
for index in np.argsort(-h)[0:5]:
  print("Node id ", index + 1, "has hubbiness scores ", h[index])
print("\n")
print("Bottom 5 node ids with hubbiness scores:")
for index in np.argsort(h)[0:5]:
  print("Node id ", index + 1, "has hubbiness scores ", h[index])

Top 5 node ids with hubbiness scores:
Node id  840 has hubbiness scores  1.0
Node id  155 has hubbiness scores  0.9499618624906541
Node id  234 has hubbiness scores  0.8986645288972266
Node id  389 has hubbiness scores  0.863417110184379
Node id  472 has hubbiness scores  0.863284109249522


Bottom 5 node ids with hubbiness scores:
Node id  23 has hubbiness scores  0.042066854890936534
Node id  835 has hubbiness scores  0.05779059354433016
Node id  141 has hubbiness scores  0.0645311764622518
Node id  539 has hubbiness scores  0.06602659373418493
Node id  889 has hubbiness scores  0.07678413939216454


In [13]:
print("Top 5 node ids with authority scores:")
for index in np.argsort(-a)[0:5]:
  print("Node id ", index + 1, "has authority scores ", a[index])
print("\n")
print("Bottom 5 node ids with authority scores:")
for index in np.argsort(a)[0:5]:
  print("Node id ", index + 1, "has authority scores ", a[index])

Top 5 node ids with authority scores:
Node id  893 has authority scores  1.0
Node id  16 has authority scores  0.9635572849634397
Node id  799 has authority scores  0.9510158161074016
Node id  146 has authority scores  0.9246703586198443
Node id  473 has authority scores  0.8998661973604049


Bottom 5 node ids with authority scores:
Node id  19 has authority scores  0.056083163776076174
Node id  135 has authority scores  0.06653910487622795
Node id  462 has authority scores  0.075442286246419
Node id  24 has authority scores  0.08171239406816945
Node id  910 has authority scores  0.08571673456144877
